# ComfyUI 模型下载工具


本笔记本用于下载和管理 ComfyUI 所需的各类模型。所有步骤以**成功挂载 Google Drive** 为前提，默认目录 `/content/drive/MyDrive/ComfyUI/models/`。


## 🌟 特性
- **完全独立运行**: 无需先安装 ComfyUI，可直接下载模型到 Google Drive（Drive 必须挂载）
- **集中配置**: 所有模型链接在第4步单独配置
- **持久化存储**: 模型保存在 `/content/drive/MyDrive/ComfyUI/models/`
- **自动跳过**: 已下载的模型不会重复下载


## ▶️ 运行顺序
1. **第1步**: 挂载 Google Drive（必跑，失败即停）
2. **第1.5步**: (可选) 配置 HF Token
3. **第2步**: 配置目录
4. **第3步**: 加载下载函数
5. **第4步**: ⭐ **配置模型列表**（在这里添加/删除模型）
6. **第5步**: 执行下载
7. **第6步**: 验证结果

## 第1步：Colab 环境初始化和 Google Drive 挂载

In [ ]:
# ============================================================
# Google Drive 挂载（Colab 专用，必跑）
# ============================================================

from google.colab import drive
import sys
import os
from pathlib import Path

print("🔗 挂载 Google Drive...")
try:
    drive.mount('/content/drive', force_remount=False)
    print("✓ Google Drive 已挂载到 /content/drive")
except Exception as e:
    print(f"✗ Drive 挂载失败: {e}")
    print("请在上方弹出的认证框中登录并授权访问 Google Drive")
    raise SystemExit("❌ 挂载失败，已终止后续步骤")

drive_root = Path('/content/drive/MyDrive')
if not drive_root.exists():
    raise SystemExit("❌ 未检测到 /content/drive/MyDrive，请确认挂载成功后重试")
print("\n" + "=" * 70)

## 第1.5步：配置 Hugging Face Token (解决 401 错误)

部分模型需要 Hugging Face 认证才能下载。
1. 访问 [Hugging Face Tokens](https://huggingface.co/settings/tokens) 获取 Token (权限选 Read)。
2. 运行下方单元格并粘贴 Token。

In [ ]:
import getpass

print("🔑 配置 Hugging Face Token")
print("------------------------------------------------------------")

# 尝试从 Colab Secrets 读取
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✓ 从 Colab Secrets 读取到 HF_TOKEN")
except Exception:
    HF_TOKEN = None

# 手动输入
if not HF_TOKEN:
    print("请输入 Hugging Face Access Token (输入时不可见，输完按回车):")
    HF_TOKEN = getpass.getpass("Token: ").strip()

if HF_TOKEN:
    print("✓ Token 已配置，将用于 Hugging Face 下载")
else:
    print("⚠ 未配置 Token。如果下载报错 401 Unauthorized，请重新运行此单元格并输入 Token。")

## 第2步：配置模型目录和下载源（Colab 专用）

In [ ]:
# ============================================
# 配置 Google Drive 模型目录
# ============================================
# 本单元格只负责目录配置，模型列表在第4步定义
# ============================================

from pathlib import Path
import os

def verify_drive_writable(path: Path) -> bool:
    """验证 Drive 路径是否真正可写。"""
    test_file = path / '.write_test'
    try:
        test_file.write_text('test')
        test_file.unlink()
        return True
    except Exception as e:
        print(f"  ⚠ Drive 写入测试失败: {e}")
        return False

print("=" * 70)
print("📂 配置 Google Drive 模型目录")
print("=" * 70)

# 检查 Drive 挂载
drive_root = Path('/content/drive/MyDrive')

if not drive_root.exists():
    print("\n❌ 错误: Google Drive 未挂载!")
    print("   请先运行第1步的 Drive 挂载单元格")
    MODELS_BASE = None
else:
    print(f"\n✓ Drive 已挂载: {drive_root}")
    
    # 创建目录结构
    drive_comfy = drive_root / 'ComfyUI'
    MODELS_BASE = drive_comfy / 'models'
    
    drive_comfy.mkdir(parents=True, exist_ok=True)
    MODELS_BASE.mkdir(parents=True, exist_ok=True)
    
    # 验证可写性
    if not verify_drive_writable(MODELS_BASE):
        print("\n❌ 错误: Drive 不可写!")
        MODELS_BASE = None
    else:
        print(f"✓ Drive 可写性验证通过")
        
        # 创建所有模型子目录
        model_subdirs = [
            'checkpoints', 'vae', 'clip', 'clip_vision', 'controlnet', 
            't2i_adapter', 'loras', 'embeddings', 'upscale_models', 
            'gligen', 'ipadapter', 'text_encoders', 'diffusion_models',
            'image_encoders', 'audio_encoders', 'unet', 'configs', 'taesd'
        ]
        
        for subdir in model_subdirs:
            (MODELS_BASE / subdir).mkdir(parents=True, exist_ok=True)
        
        print(f"✓ 已创建 {len(model_subdirs)} 个模型子目录")
        print(f"\n✅ 模型目录: {MODELS_BASE}")

if MODELS_BASE is None:
    raise SystemExit("❌ 未成功配置 Drive 模型目录，已终止后续步骤")

## 第3步：下载函数库

In [ ]:
# ============================================
# 下载函数库
# ============================================

import urllib.request
import urllib.error
from pathlib import Path

def download_file(url: str, output_path: Path, timeout: int = 300) -> bool:
    """
    下载单个文件到指定路径。
    
    Args:
        url: 文件 URL
        output_path: 输出文件路径
        timeout: 超时时间（秒）
    
    Returns:
        成功返回 True，失败返回 False
    """
    try:
        print(f"  ⏳ 下载中: {url}")
        
        # 构建请求，如果配置了 Token 且是 HF 链接，则添加认证头
        req = urllib.request.Request(url)
        if 'HF_TOKEN' in globals() and HF_TOKEN and 'huggingface.co' in url:
            req.add_header('Authorization', f'Bearer {HF_TOKEN}')
        
        # 使用 urlopen 替代 urlretrieve 以支持 timeout 和 headers
        with urllib.request.urlopen(req, timeout=timeout) as response:
            total_size = int(response.info().get('Content-Length', -1))
            downloaded = 0
            chunk_size = 1024 * 1024  # 1MB 分块读取
            
            with open(output_path, 'wb') as out_file:
                while True:
                    chunk = response.read(chunk_size)
                    if not chunk:
                        break
                    out_file.write(chunk)
                    downloaded += len(chunk)
                    
                    # 进度显示
                    if total_size > 0:
                        percent = downloaded / total_size * 100
                        print(f"    ⬇️  {percent:.1f}% ({downloaded/(1024**2):.1f}/{total_size/(1024**2):.1f} MB)", end='\r')
                    else:
                        print(f"    ⬇️  {downloaded/(1024**2):.1f} MB", end='\r')
        
        print() # 换行
        file_size = output_path.stat().st_size / (1024 ** 2)  # MB
        print(f"  ✓ 下载完成: {output_path.name} ({file_size:.2f} MB)")
        return True
    except urllib.error.HTTPError as e:
        print(f"\n  ✗ HTTP 错误 {e.code}: {e.reason}")
        if e.code == 401:
            print("    提示: 请检查是否已在第1.5步配置了正确的 Hugging Face Token")
        return False
    except urllib.error.URLError as e:
        print(f"\n  ✗ URL 错误: {e.reason}")
        return False
    except Exception as e:
        print(f"\n  ✗ 下载失败: {e}")
        return False


def download_model_with_fallback(
    model_name: str, 
    urls: list, 
    output_path: Path,
    force: bool = False
) -> bool:
    """
    尝试从多个 URL 源下载模型（故障转移）。
    
    Args:
        model_name: 模型名称
        urls: URL 列表
        output_path: 输出路径
        force: 强制重新下载（即使文件已存在）
    
    Returns:
        成功返回 True，全部失败返回 False
    """
    # 如果文件已存在且不强制重新下载，跳过
    if output_path.exists() and not force:
        file_size = output_path.stat().st_size / (1024 ** 2)
        print(f"✓ {model_name} 已存在 ({file_size:.2f} MB)，跳过")
        return True
    
    # 从不同源尝试下载
    for i, url in enumerate(urls, 1):
        print(f"  [源 {i}/{len(urls)}]")
        
        # 临时文件，下载失败时自动删除
        temp_file = output_path.with_suffix(output_path.suffix + '.tmp')
        if temp_file.exists():
            temp_file.unlink()
        
        if download_file(url, temp_file):
            # 下载成功，重命名为目标文件
            temp_file.rename(output_path)
            return True
        else:
            # 下载失败，删除临时文件
            if temp_file.exists():
                temp_file.unlink()
            if i < len(urls):
                print(f"  将尝试下一个源...\n")
    
    print(f"✗ {model_name} 所有源均下载失败\n")
    return False


print("✅ 下载函数已加载")
print("   请继续运行第4步配置模型列表")

## 第4步：配置模型下载列表

在下方单元格中编辑 `MODELS_CONFIG`，添加或删除需要下载的模型。

### 📋 可用模型说明

#### 🌟 Qwen Image Edit / Flux2（推荐，默认启用）
| 模型名称 | 类型 | 说明 |
|----------|------|------|
| **qwen_image_edit_fp8_e4m3fn** | diffusion_models | Qwen 图像编辑扩散模型 (FP8量化) |
| **flux2_dev_fp8mixed** | diffusion_models | Flux2 开发版扩散模型 (FP8混合精度) |
| **stable-diffusion-v1-5** | diffusion_models | SD 1.5 官方扩散模型 (pruned版本) |
| **qwen_2.5_vl_7b_fp8_scaled** | text_encoders | Qwen 2.5 视觉语言文本编码器 (7B, FP8) |
| **mistral_3_small_flux2_bf16** | text_encoders | Mistral 3 Small Flux2 文本编码器 (BF16) |
| **qwen_image_vae** | vae | Qwen Image 专用 VAE |
| **flux2-vae** | vae | Flux2 专用 VAE |
| **chilloutmix_NiPrunedFp32Fix** | vae | ChilloutMix 专用 VAE (真实感优化) |
| **Qwen-Image-Lightning-4steps** | loras | Qwen Image 加速 LoRA (4步推理) |
| **koreanDollLikeness** | loras | 韩国娃娃脸风格 LoRA |
| **Cute_girl_mix4** | loras | 可爱女孩风格 LoRA |
| **Nudify_XL_Better_Bodies** | loras | SDXL 人体优化 LoRA |

#### 🎨 Checkpoints（主模型）
| 模型名称 | 说明 | 大小 | 适用场景 |
|----------|------|------|----------|
| **sd_xl_base_1.0** | SDXL 基础模型 | ~6.5GB | 高质量 1024x1024 图像生成 |
| **sd_xl_refiner_1.0** | SDXL 精修模型 | ~6.0GB | 配合 base 使用，提升细节 |
| **v1-5-pruned-emaonly** | SD 1.5 经典模型 | ~4.0GB | 512x512，资源需求低，生态丰富 |
| **v2-1_512-ema-pruned** | SD 2.1 512 版本 | ~5.0GB | 512x512，质量优于 1.5 |
| **v2-1_768-ema-pruned** | SD 2.1 768 版本 | ~5.0GB | 768x768，更高分辨率 |
| **AbyssOrangeMix2/3** | 动漫风格模型 | ~2.0GB | 二次元、动漫插画 |
| **anything-v3** | 万能动漫模型 | ~2.0GB | 泛用动漫风格 |
| **wd-illusion** | Waifu Diffusion 1.5 | ~2.0GB | 高质量动漫，SD2 架构 |

#### 🖼️ VAE（变分自编码器）
| 模型名称 | 说明 | 适用场景 |
|----------|------|----------|
| **vae-ft-mse-840000** | 官方 MSE 微调 VAE | 通用，减少色彩失真 |
| **chilloutmix_NiPrunedFp32Fix** | ChilloutMix VAE | 真实感人像优化 |
| **orangemix.vae** | OrangeMix 专用 VAE | 动漫风格优化 |
| **kl-f8-anime2** | 动漫专用 VAE | Waifu Diffusion 系列 |

#### 🔗 LoRA（轻量微调）
| 模型名称 | 说明 | 来源 |
|----------|------|------|
| **Qwen-Image-Lightning-4steps** | Qwen 加速推理 | HuggingFace |
| **koreanDollLikeness** | 韩国娃娃脸风格，精致五官 | Civitai |
| **Cute_girl_mix4** | 可爱女孩风格混合 | Civitai |
| **Nudify_XL_Better_Bodies** | SDXL 人体结构优化 | Civitai |
| **theovercomer8sContrastFix** | 对比度修复，提升画面层次感 | Civitai |
| **sd_xl_offset_example-lora** | SDXL 噪声偏移，改善暗部/亮部细节 | HuggingFace |

#### 🎛️ ControlNet（控制网络）
| 模型名称 | 控制类型 | 说明 |
|----------|----------|------|
| **control_v11p_sd15_canny** | 边缘检测 | 从线稿生成图像 |
| **control_v11p_sd15_depth** | 深度图 | 控制图像空间结构 |
| **control_v11p_sd15_openpose** | 人体姿态 | 控制人物动作 |
| **control_v11p_sd15_scribble** | 涂鸦 | 从草图生成 |
| **control_v11p_sd15_lineart** | 线稿 | 精细线条控制 |
| **control_v11p_sd15_softedge** | 软边缘 | 柔和的边缘引导 |
| **control_v11p_sd15_seg** | 语义分割 | 按区域控制内容 |
| **control_v11p_sd15_inpaint** | 局部重绘 | 图像修复/扩展 |
| **control_v11u_sd15_tile** | 超分/细节 | 提升分辨率和细节 |

#### 🔍 CLIP Vision
| 模型名称 | 说明 |
|----------|------|
| **clip_vision_g** | SDXL ReVision 图像引导所需 |
| **clip_vit14** | T2I 样式迁移所需 |

#### 📐 T2I-Adapter（轻量控制）
| 模型名称 | 控制类型 |
|----------|----------|
| **t2iadapter_canny** | 边缘检测 |
| **t2iadapter_depth** | 深度图 |
| **t2iadapter_sketch** | 草图 |
| **t2iadapter_openpose** | 人体姿态 |
| **t2iadapter_color** | 颜色引导 |
| **t2iadapter_style** | 风格迁移 |

#### ⬆️ Upscale（超分模型）
| 模型名称 | 放大倍数 | 说明 |
|----------|----------|------|
| **RealESRGAN_x4plus** | 4x | 真实感图像超分 |
| **RealESRGAN_x2** | 2x | 2倍超分 |
| **RealESRGAN_x4** | 4x | 4倍超分 |

#### 🎯 GLIGEN（区域控制）
| 模型名称 | 说明 |
|----------|------|
| **gligen_sd14_textbox** | 使用文本框控制生成区域 |

In [ ]:
# ============================================
# 📦 模型下载配置
# ============================================
# 在这里添加/删除/修改所有要下载的模型
# 格式: '文件名': ['下载链接1', '备用链接2', ...]
# 取消注释 (#) 即可启用对应模型的下载
# ============================================

MODELS_CONFIG = {
    # ==================== Diffusion Models 扩散模型 ====================
    'diffusion_models': {
        'qwen_image_edit_fp8_e4m3fn.safetensors': [
            'https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI/resolve/main/split_files/diffusion_models/qwen_image_edit_fp8_e4m3fn.safetensors'
        ],
        'flux2_dev_fp8mixed.safetensors': [
            'https://huggingface.co/Comfy-Org/flux2-dev/resolve/main/split_files/diffusion_models/flux2_dev_fp8mixed.safetensors'
        ],
        'stable-diffusion-v1-5.safetensors': [
            'https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned.safetensors'
        ],
    },
    
    # ==================== LoRA 轻量微调模型 ====================
    'loras': {
        'Qwen-Image-Lightning-4steps-V1.0.safetensors': [
            'https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-4steps-V1.0.safetensors'
        ],
        'koreanDollLikeness.safetensors': [
            'https://civitai.com/api/download/models/31284?type=Model&format=SafeTensor&size=full&fp=fp16'
        ],
        'Cute_girl_mix4.safetensors': [
            'https://civitai.com/api/download/models/16677?type=Model&format=SafeTensor&size=full&fp=fp16'
        ],
        'Nudify_XL_Better_Bodies.safetensors': [
            'https://civitai.com/api/download/models/177674?type=Model&format=SafeTensor'
        ],
        # 'theovercomer8sContrastFix_sd21768.safetensors': [
        #     'https://civitai.com/api/download/models/10350'
        # ],
        # 'theovercomer8sContrastFix_sd15.safetensors': [
        #     'https://civitai.com/api/download/models/10638'
        # ],
        # 'sd_xl_offset_example-lora_1.0.safetensors': [
        #     'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors'
        # ],
    },
    
    # ==================== VAE 变分自编码器 ====================
    'vae': {
        'qwen_image_vae.safetensors': [
            'https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors'
        ],
        'flux2-vae.safetensors': [
            'https://huggingface.co/Comfy-Org/flux2-dev/resolve/main/split_files/vae/flux2-vae.safetensors'
        ],
        'chilloutmix_NiPrunedFp32Fix.safetensors': [
            'https://civitai.com/api/download/models/11745?type=VAE&format=Other'
        ],
        # 'vae-ft-mse-840000-ema-pruned.safetensors': [
        #     'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors'
        # ],
        # 'orangemix.vae.pt': [
        #     'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt'
        # ],
        # 'kl-f8-anime2.ckpt': [
        #     'https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt'
        # ],
    },
    
    # ==================== Text Encoders 文本编码器 ====================
    'text_encoders': {
        'qwen_2.5_vl_7b_fp8_scaled.safetensors': [
            'https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors'
        ],
        'mistral_3_small_flux2_bf16.safetensors': [
            'https://huggingface.co/Comfy-Org/flux2-dev/resolve/main/split_files/text_encoders/mistral_3_small_flux2_bf16.safetensors'
        ],
    },
    
    # ==================== Checkpoints 主模型（取消注释启用）====================
    'checkpoints': {
        # ----- SDXL 系列 (1024x1024) -----
        # 'sd_xl_base_1.0.safetensors': [
        #     'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors'
        # ],
        # 'sd_xl_refiner_1.0.safetensors': [
        #     'https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors'
        # ],
        
        # ----- SD 1.5 经典模型 (512x512) -----
        # 'v1-5-pruned-emaonly.ckpt': [
        #     'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt'
        # ],
        
        # ----- SD 2.x 系列 -----
        # 'v2-1_512-ema-pruned.safetensors': [
        #     'https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors'
        # ],
        # 'v2-1_768-ema-pruned.safetensors': [
        #     'https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors'
        # ],
        
        # ----- 动漫风格模型 -----
        # 'AbyssOrangeMix2_hard.safetensors': [
        #     'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors'
        # ],
        # 'AOM3A1_orangemixs.safetensors': [
        #     'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors'
        # ],
        # 'AOM3A3_orangemixs.safetensors': [
        #     'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors'
        # ],
        # 'anything-v3-fp16-pruned.safetensors': [
        #     'https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors'
        # ],
        # 'wd-illusion-fp16.safetensors': [
        #     'https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors'
        # ],
        
        # ----- unCLIP 模型 -----
        # 'illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors'
        # ],
        # 'wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors'
        # ],
    },
    
    # ==================== ControlNet 控制网络 (SD 1.5) ====================
    'controlnet': {
        # ----- 基础控制 -----
        # 'control_v11p_sd15_canny_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors'
        # ],
        # 'control_v11f1p_sd15_depth_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors'
        # ],
        # 'control_v11p_sd15_openpose_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors'
        # ],
        # 'control_v11p_sd15_scribble_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors'
        # ],
        
        # ----- 线稿/边缘 -----
        # 'control_v11p_sd15_lineart_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors'
        # ],
        # 'control_v11p_sd15s2_lineart_anime_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors'
        # ],
        # 'control_v11p_sd15_softedge_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors'
        # ],
        # 'control_v11p_sd15_mlsd_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors'
        # ],
        
        # ----- 高级控制 -----
        # 'control_v11p_sd15_seg_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors'
        # ],
        # 'control_v11p_sd15_normalbae_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors'
        # ],
        # 'control_v11p_sd15_inpaint_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors'
        # ],
        # 'control_v11u_sd15_tile_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors'
        # ],
        # 'control_v11e_sd15_shuffle_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors'
        # ],
        # 'control_v11e_sd15_ip2p_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors'
        # ],
        
        # ----- ControlNet SDXL -----
        # 'control-lora-canny-rank256.safetensors': [
        #     'https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors'
        # ],
        # 'control-lora-depth-rank256.safetensors': [
        #     'https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors'
        # ],
        # 'control-lora-recolor-rank256.safetensors': [
        #     'https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors'
        # ],
        # 'control-lora-sketch-rank256.safetensors': [
        #     'https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors'
        # ],
    },
    
    # ==================== CLIP Vision 视觉编码器 ====================
    'clip_vision': {
        # 'clip_vision_g.safetensors': [
        #     'https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors'
        # ],
        # 'clip_vit14.bin': [
        #     'https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin'
        # ],
    },
    
    # ==================== CLIP 模型 ====================
    'clip': {
        # 添加 CLIP 模型...
    },
    
    # ==================== T2I-Adapter 轻量控制 ====================
    't2i_adapter': {
        # 't2iadapter_canny_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth'
        # ],
        # 't2iadapter_depth_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth'
        # ],
        # 't2iadapter_sketch_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth'
        # ],
        # 't2iadapter_openpose_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth'
        # ],
        # 't2iadapter_keypose_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth'
        # ],
        # 't2iadapter_seg_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth'
        # ],
        # 't2iadapter_color_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth'
        # ],
    },
    
    # ==================== Style Models 风格模型 ====================
    'style_models': {
        # 't2iadapter_style_sd14v1.pth': [
        #     'https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth'
        # ],
    },
    
    # ==================== GLIGEN 区域控制 ====================
    'gligen': {
        # 'gligen_sd14_textbox_pruned_fp16.safetensors': [
        #     'https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors'
        # ],
    },
    
    # ==================== Upscale 超分模型 ====================
    'upscale_models': {
        # 'RealESRGAN_x4plus.pth': [
        #     'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth'
        # ],
        # 'RealESRGAN_x2.pth': [
        #     'https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth'
        # ],
        # 'RealESRGAN_x4.pth': [
        #     'https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth'
        # ],
    },
}

# ============================================
# 显示配置的模型
# ============================================
print("📋 已配置的模型 (将会下载):")
print("=" * 50)
total_count = 0
for category, models in MODELS_CONFIG.items():
    if models:
        print(f"\n📁 {category}/")
        for name in models.keys():
            print(f"   • {name}")
            total_count += 1

if total_count == 0:
    print("\n⚠ 没有配置任何模型，请取消注释需要的模型")
else:
    print(f"\n📊 共计 {total_count} 个模型将被下载")

## 第5步：执行下载

In [ ]:
# ============================================
# 执行模型下载
# ============================================

print("=" * 70)
print("🚀 开始下载模型")
print("=" * 70)

# 检查必要条件
if 'MODELS_BASE' not in globals() or MODELS_BASE is None:
    print("❌ MODELS_BASE 未配置，请先运行第2步")
elif 'MODELS_CONFIG' not in globals():
    print("❌ MODELS_CONFIG 未定义，请先运行第4步")
elif 'download_model_with_fallback' not in globals():
    print("❌ 下载函数未加载，请先运行第3步")
else:
    # 统计要下载的模型
    total_models = sum(len(m) for m in MODELS_CONFIG.values() if m)
    print(f"\n📋 待下载模型数: {total_models}")
    
    # 下载
    results = {}
    for category, models in MODELS_CONFIG.items():
        if not models:
            continue
            
        category_dir = MODELS_BASE / category
        category_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n📦 {category}/")
        print("-" * 50)
        
        for model_name, urls in models.items():
            output_path = category_dir / model_name
            results[model_name] = download_model_with_fallback(
                model_name, urls, output_path, force=False
            )

    # 汇总
    if results:
        print("\n" + "=" * 70)
        print("📊 下载结果:")
        print("=" * 70)
        
        success_count = sum(1 for v in results.values() if v)
        
        for model_name, success in results.items():
            print(f"  {'✓' if success else '✗'} {model_name}")
        
        print(f"\n总计: {success_count}/{len(results)} 成功")
        
        if success_count == len(results):
            print("\n✅ 所有模型下载完成！")
        else:
            print("\n⚠ 部分失败，检查网络或 HF Token 后重试")
        
        print(f"\n📂 保存位置: {MODELS_BASE}")
    else:
        print("\n⚠ 没有配置任何模型，请在第4步的 MODELS_CONFIG 中添加")

## 第6步：验证已下载的模型

In [ ]:
# ============================================
# 验证 Google Drive 上的模型文件
# ============================================

import os
from pathlib import Path
import subprocess

print("=" * 70)
print("🔍 验证 Google Drive 模型文件")
print("=" * 70)

# Drive 模型目录
drive_models = Path('/content/drive/MyDrive/ComfyUI/models')

if not drive_models.exists():
    print("\n❌ Drive 模型目录不存在!")
    print(f"   路径: {drive_models}")
    print("   请先运行第2步配置单元格")
else:
    print(f"\n📂 模型目录: {drive_models}")
    
    # 1. 验证已配置的模型
    print("\n" + "-" * 70)
    print("📦 已配置模型检查:")
    print("-" * 70)
    
    total_size = 0
    all_exist = True
    
    if 'MODELS_CONFIG' in globals():
        for category, models in MODELS_CONFIG.items():
            if not models:
                continue
            category_dir = drive_models / category
            print(f"\n{category}/")
            
            for model_name in models.keys():
                model_path = category_dir / model_name
                
                if model_path.exists():
                    size_mb = model_path.stat().st_size / (1024 ** 2)
                    total_size += model_path.stat().st_size
                    print(f"  ✓ {model_name} ({size_mb:.2f} MB)")
                else:
                    print(f"  ✗ {model_name} (未找到)")
                    all_exist = False
    else:
        print("⚠ MODELS_CONFIG 未定义，请先运行第4步")
    
    # 2. 扫描 Drive 上所有模型
    print("\n" + "-" * 70)
    print("📋 Drive 完整模型目录:")
    print("-" * 70)
    
    drive_total = 0
    for category_dir in sorted(drive_models.iterdir()):
        if category_dir.is_dir():
            files = list(category_dir.glob('*.safetensors')) + list(category_dir.glob('*.ckpt'))
            if files:
                category_size = sum(f.stat().st_size for f in files)
                drive_total += category_size
                print(f"\n{category_dir.name}/ ({category_size/(1024**3):.2f} GB)")
                for f in sorted(files):
                    size_mb = f.stat().st_size / (1024 ** 2)
                    print(f"  ├─ {f.name} ({size_mb:.2f} MB)")
    
    if drive_total == 0:
        print("\n(空 - 尚未下载任何模型)")
    
    # 3. 汇总
    print("\n" + "=" * 70)
    print("📊 汇总:")
    print("=" * 70)
    
    total_size_gb = total_size / (1024 ** 3) if total_size > 0 else 0
    drive_total_gb = drive_total / (1024 ** 3) if drive_total > 0 else 0
    
    print(f"  已配置模型: {total_size_gb:.2f} GB")
    print(f"  Drive 总计: {drive_total_gb:.2f} GB")
    
    if 'MODELS_CONFIG' in globals() and all_exist:
        print("\n  ✅ 所有配置的模型已下载到 Google Drive!")
    elif 'MODELS_CONFIG' in globals():
        print("\n  ⚠ 部分模型缺失，请运行第5步下载")